In [2]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

musings_path = 'sam_musings/'
musings = os.listdir(musings_path)

# Import Dataset
print('Loading musings...')
df = pd.DataFrame({'content': [open(musings_path + musing).read() for musing in musings][0:10]})

# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
print('Making bigrams...')
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

print('Making trigrams...')
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
print('Modding n-grams...')
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
print('Destroying stop words...')
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
print('Forming bigrams...')
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
print('Building spacy NLP parser...')
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
print('Lemmatizing bigrams...')
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
print('Building dictionary...')
id2word = corpora.Dictionary(data_lemmatized)
id2word.save('sam_dictionary.dict')

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
print('Making corpus...')
corpus = [id2word.doc2bow(text) for text in texts]

topic_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 200]
lda_models = [] * len(topic_sizes)
coherence_model_ldas = [] * len(topic_sizes)
coherence_lda = np.zeros(len(topic_sizes))

# Build LDA model
print('Generating LDA Model...')
#for index, topics in enumerate(topic_sizes):
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=30, 
                                                   random_state=100,
                                                   update_every=1,
                                                   chunksize=100,
                                                   passes=10,
                                                   alpha='auto',
                                                   per_word_topics=True)
    
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Generate mallet topic
mallet_path = '../mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=32, limit=40, step=6)

# Show graph
limit=40; start=32; step=6;
x = range(start, limit, step)

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

# Select the model and print the topics
optimal_model = model_list[np.argmax(coherence_values)]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


Loading musings...
Making bigrams...
Making trigrams...
Modding n-grams...


/home/zakeliza/.local/lib/python2.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Destroying stop words...
Forming bigrams...
Building spacy NLP parser...
Lemmatizing bigrams...
Building dictionary...
Making corpus...
Generating LDA Model...
('\nPerplexity: ', -7.346975746712915)
('\nCoherence Score: ', 0.3410900764925528)
('Num Topics =', 32, ' has Coherence Value of', 0.5121)
('Num Topics =', 38, ' has Coherence Value of', 0.5266)
[(25,
  u'0.092*"marcus" + 0.057*"discover" + 0.057*"editor" + 0.046*"literary" + 0.034*"marsha" + 0.034*"original" + 0.034*"teenager" + 0.023*"teenage" + 0.023*"library" + 0.023*"ratify"'),
 (10,
  u'0.321*"write" + 0.131*"request" + 0.071*"topic" + 0.036*"essay" + 0.036*"person" + 0.024*"girl" + 0.024*"eyed" + 0.024*"lover" + 0.024*"potential" + 0.024*"morrison"'),
 (21,
  u'0.123*"young" + 0.070*"spend" + 0.053*"iowa" + 0.053*"suggest" + 0.035*"day" + 0.035*"future" + 0.035*"assumption" + 0.035*"easy" + 0.035*"central" + 0.035*"bad"'),
 (2,
  u'0.143*"case" + 0.054*"reaction" + 0.054*"enjoy" + 0.054*"back" + 0.036*"copy" + 0.036*"bruc

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,26.0,0.2515,"work, job, worth, question, answer, control, t...","Toward the end of today's CSC 151, a student a..."
1,1,8.0,0.4217,"musing, page, canonical, url, annotation, hypo...","Theme/topics: misc, the musings, bad coding ha..."
2,2,25.0,0.1259,"marcus, discover, editor, literary, marsha, or...","As part of my mix ""tape"" project, I've been re..."
3,3,4.0,0.1886,"kumail, main, graduate, remember, professional...",A much delayed reflection. As many of you know...
4,4,10.0,0.2951,"write, request, topic, essay, person, girl, ey...","Since I've started writing these essays, some ..."
5,5,1.0,0.1907,"live, year, love, put, wonderful, skill, weddi...","Thirty years ago today, Michelle Steele (Rebel..."
6,6,18.0,0.4020,"message, inbox, thing, move, email, mail, app,...",Given all of the problems I've had with moving...
7,7,13.0,0.2334,"faculty, diversity, committee, curriculum, mem...","Themes/tags: Grinnell, rants, diversity, writi..."
8,8,11.0,0.3484,"issue, dean, support, concern, position, depar...","I'm a curmudgeon. You should know that by now,..."
9,9,27.0,0.4429,"coffee, shop, knock, university, chicago, divi...","More than 30 years ago, I bought the mug that ..."


In [3]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,1.0,0.1907,"live, year, love, put, wonderful, skill, weddi...","Thirty years ago today, Michelle Steele (Rebel..."
1,4.0,0.1886,"kumail, main, graduate, remember, professional...",A much delayed reflection. As many of you know...
2,8.0,0.4217,"musing, page, canonical, url, annotation, hypo...","Theme/topics: misc, the musings, bad coding ha..."
3,10.0,0.2951,"write, request, topic, essay, person, girl, ey...","Since I've started writing these essays, some ..."
4,11.0,0.3484,"issue, dean, support, concern, position, depar...","I'm a curmudgeon. You should know that by now,..."


In [4]:
sent_topics_sorteddf_mallet['Text'][1]

'A much delayed reflection. As many of you know, Kumail Nanjiani \'01\'s movie, The Big Sick was recently released. I very much appreciate the movie, on many levels, and for many reasons. Here are some. It\'s nice to see a romcom [1] in which the male lead represents a non-majority group. It\'s particularly nice to see that when our country is in its current state. It reminds me of how happy Kumail is with Emily. I really like seeing my students happy. It does not reference his summer doing research with me. He used to reference it in one of his shows. If I recall correctly, it went something like the following. Then I got hired to do summer research. What I mostly recall about that summer is sitting under a tree while smoking cigarettes, contemplating philosophy, and losing my religion [2,9]. When Kumail was on campus to give his legendary graduation speech, I gave Emily one of the amazing Grinnell Women in Computing shirts [10,11], which has a brain [12] superimposed over a grid of b

In [27]:
def getTopicForQuery(question):
    temp = question.lower()

    words = re.findall(r'\w+', temp, flags = re.UNICODE | re.LOCALE)

    important_words = remove_stopwords(words)
    print important_words

    dictionary = corpora.Dictionary.load('sam_dict.dict')

    ques_vec = [dictionary.doc2bow(text) for text in texts]
    
    # load LDA model
    topic_vec = lda_model[ques_vec]

    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    print idx
    word_count_array = word_count_array[idx]
    print word_count_array
    
    final = []
    final = lda_model.print_topic(word_count_array[0, 0], 1)

    question_topic = final.split('*') ## as format is like "probability * topic"

    return question_topic[1]

In [28]:
getTopicForQuery('Hi hecker.')

[[u'hi'], [u'hecker']]
[9 5 4 1 6 3 2 8 7 0]
[[list([(2, 0.9942752)])
  list([(39, [2]), (43, [2]), (45, [2]), (53, [2]), (57, [2]), (68, [2]), (69, [2]), (76, [2]), (81, [2]), (82, [2]), (101, [2]), (106, [2]), (125, [2]), (139, [2]), (156, [2]), (173, [2]), (223, [2]), (232, [2]), (234, [2]), (235, [2]), (243, [2]), (259, [2]), (275, [2]), (291, [2]), (298, [2]), (303, [2]), (306, [2]), (309, [2]), (345, [2]), (353, [2]), (371, [2]), (381, [2]), (405, [2]), (492, [2]), (502, [2]), (528, [2]), (542, [2]), (629, [2]), (669, [2]), (676, [2]), (686, [2]), (696, [2]), (740, [2]), (742, [2]), (756, [2]), (777, [2]), (786, [2]), (802, [2]), (829, [2]), (836, [2]), (837, [2]), (861, [2]), (879, [2]), (1009, [2]), (1036, [2]), (1043, [2]), (1077, [2]), (1086, [2]), (1169, [2]), (1180, [2]), (1209, [2]), (1243, [2]), (1245, [2]), (1337, [2]), (1338, [2]), (1339, [2]), (1340, [2]), (1341, [2]), (1342, [2]), (1343, [2]), (1344, [2]), (1345, [2]), (1346, [2]), (1347, [2]), (1348, [2]), (1349, [2]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
id2word.save('sam_dict.dict')

In [15]:
data_words_nostops

[[u'toward',
  u'end',
  u'today',
  u'csc',
  u'student',
  u'asked',
  u'something',
  u'like',
  u'following',
  u'sam',
  u'understand',
  u'bunches',
  u'advisees',
  u'answer',
  u'questions',
  u'quickly',
  u'via',
  u'email',
  u'even',
  u'though',
  u'lots',
  u'students',
  u'seem',
  u'spend',
  u'time',
  u'family',
  u'write',
  u'musing',
  u'every',
  u'night',
  u'since',
  u'reviewing',
  u'final',
  u'want',
  u'give',
  u'long',
  u'answer',
  u'gave',
  u'short',
  u'one',
  u'lot',
  u'work',
  u'worth',
  u'get',
  u'chance',
  u'work',
  u'awesome',
  u'students',
  u'sometimes',
  u'make',
  u'positive',
  u'contribution',
  u'easily',
  u'worth',
  u'true',
  u'like',
  u'work',
  u'bit',
  u'less',
  u'expect',
  u'find',
  u'way',
  u'eventually',
  u'job',
  u'incredibly',
  u'rewarding',
  u'also',
  u'moderate',
  u'control',
  u'choose',
  u'beyond',
  u'core',
  u'control',
  u'teach',
  u'plus',
  u'love',
  u'teaching',
  u'programming',
  u'curmudge